In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool, BaseTool, DuckDuckGoSearchResults
from pydantic import BaseModel, Field
from typing import Type
import requests
import yfinance as yf

In [2]:
llm = ChatOpenAI(temperature=0.1, model_name="gpt-4o")

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a compnay.
    It takes a query as an argument.
    Example query: Stock Market Symbol for Apple Company
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        return ddg.run(query)

class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(description="Stock symbol of the company. Example: AAPL, TSLA")

class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    Overview includes news, income statement, balance sheet and cashflow of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        ticker = yf.Ticker(symbol)

        report = {
            "news": ticker.news,
            "income_statement": ticker.income_stmt,
            "balance_sheet": ticker.balance_sheet,
            "cashflow": ticker.cashflow,
        }
        return report

In [3]:
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
    ]
)

prompt = "Give me information on {company}'s stock and help me analyze if it's a potential good investment. You should refer to information which includes news, income statement, balance sheet and cashflow of the company. Also tell me what symbol does the stock have."

agent.invoke(prompt.format(company="Cloudflare"))



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare'}`


[snippet: Following its initial announcement in September 2022, Cloudflare Calls is now in open beta and available in your Cloudflare Dashboard.Cloudflare Calls lets developers build real-time audio/video apps using WebRTC, and it abstracts away the complexity by turning the Cloudflare network into a singular SFU.In this post, we dig into how we make this possible., title: Cloudflare Calls: millions of cascading trees all the way down, link: https://blog.cloudflare.com/cloudflare-calls-anycast-webrtc], [snippet: Read the latest posts from Cloudflare's official blog, covering topics such as security, DNS, DDoS, email, AI, and more. Learn about Cloudflare's new features, acquisitions, partnerships, and events., title: The Cloudflare Blog, link: https://blog.cloudflare.com/], [snippet: Finally, because Cloudflare also operates a CDN, websites that are already on Cloudflare wil

{'input': "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. You should refer to information which includes news, income statement, balance sheet and cashflow of the company. Also tell me what symbol does the stock have.",
 'output': "### Cloudflare (NET) Stock Overview\n\n**Stock Symbol:** NET\n\n### Recent News\n1. **[Why Investors Need to Take Advantage of These 2 Computer and Technology Stocks Now](https://finance.yahoo.com/news/why-investors-advantage-2-computer-130008598.html)** - Zacks\n2. **[Charge Up Your Stock Chart Analysis By Studying Volume In Bases](https://finance.yahoo.com/m/a5138c64-d919-3e02-9e9a-8c027ce634f5/charge-up-your-stock-chart.html)** - Investor's Business Daily\n3. **[Here is What to Know Beyond Why Cloudflare, Inc. (NET) is a Trending Stock](https://finance.yahoo.com/news/know-beyond-why-cloudflare-inc-130015013.html)** - Zacks\n4. **[Cloudflare Insider Sold Shares Worth $2,196,419, According to a Recent SEC 

In [4]:
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

In [ ]:
llm = ChatOpenAI(
    temperature=0.1,
)

db = SQLDatabase.from_uri("sqlite:///movies.sqlite")

toolkit = SQLDatabaseToolkit(
    db=db,
    llm=llm
)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent.invoke("Give me the 5 directors that have the highest grossing films.")